###### AUTOMATIC DIFFERENTIATION WITH <code>torch.autograd</code>

When training neural networks, the most frequently used algorithm is back propagation. In this algorithm, parameters (model weights) are adjusted according to the gradient of the loss function with respect to the given parameter.

To compute those gradients, PyTorch has a built-in differentiation engine called torch.autograd. It supports automatic computation of gradient for any computational graph.

Consider the simplest one-layer neural network, with input x, parameters w and b, and some loss function. It can be defined in PyTorch in the following manner:

In [1]:
import torch

In [2]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

__Tensors, Functions and Computational graph__

In this network, w and b are parameters, which we need to optimize. Thus, we need to be able to compute the gradients of loss function with respect to those variables. In order to do that, we set the <code>requires_grad</code> property of those tensors.

__*You can set the value of <code>requires_grad</code> when creating a tensor, or later by using <code>x.requires_grad_(True)</code> method.__

![computational graph](imgs/comp-graph.png)

A function that we apply to tensors to construct computational graph is in fact an object of class <code>Function</code>. This object knows how to compute the function in the forward direction, and also how to compute its derivative during the backward propagation step. A reference to the backward propagation function is stored in <code>grad_fn</code> property of a tensor. You can find more information of <code>Function</code> in the [documentation](https://pytorch.org/docs/stable/autograd.html#function).

In [3]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f88d4c85340>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f88d4c853a0>


__Computing Gradients__

To optimize weights of parameters in the neural network, we need to compute the derivatives of our loss function with respect to parameters, namely, we need __∂loss/∂w__ and __∂loss/∂b__ under some fixed values of <code>x</code> and <code>y</code>.


To compute those derivatives, we call <code>loss.backward()</code>, and then retrieve the values from <code>w.grad</code> and <code>b.grad</code>:

 

In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0018, 0.3236, 0.1107],
        [0.0018, 0.3236, 0.1107],
        [0.0018, 0.3236, 0.1107],
        [0.0018, 0.3236, 0.1107],
        [0.0018, 0.3236, 0.1107]])
tensor([0.0018, 0.3236, 0.1107])


We can only obtain the <code>grad</code> properties for the leaf nodes of the computational graph, which have <code>requires_grad</code> property set to <code>True</code>. For all other nodes in our graph, gradients will not be available.

We can only perform gradient calculations using <code>backward</code> once on a given graph, for performance reasons. If we need to do several <code>backward</code> calls on the same graph, we need to pass <code>retain_graph=True</code> to the <code>backward</code> call.

__Disabling Gradient Tracking__

By default, all tensors with <code>requires_grad=True</code> are tracking their computational history and support gradient computation. However, there are some cases when we do not need to do that, for example, when we have trained the model and just want to apply it to some input data, i.e. we only want to do forward computations through the network. We can stop tracking computations by surrounding our computation code with <code>torch.no_grad()</code> block:

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [6]:
# Another way to achieve the same result is to use the detach() method on the tensor:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


__There are reasons you might want to disable gradient tracking:__
<ul>
<li>To mark some parameters in your neural network as frozen parameters. This is a very common scenario for <b>finetuning a pretrained network</b> </li>
<li>To speed up computations when you are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.</li>
</ul>

__More on Computational Graphs__

Conceptually, autograd keeps a record of data (tensors) and all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of Function objects. In this DAG, leaves are the input tensors, roots are the output tensors. By tracing this graph from roots to leaves, you can automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:
<ul>
<li>run the requested operation to compute a resulting tensor</li>
<li>maintain the operation’s gradient function in the DAG.</li>
</ul>

The backward pass kicks off when <code>.backward()</code> is called on the DAG root. autograd then:
<ul>
<li>computes the gradients from each <code>.grad_fn</code>,</li>
<li>accumulates them in the respective tensor’s <code>.grad</code> attribute</li>
<li>using the chain rule, propagates all the way to the leaf tensors.</li>
</ul>

__Note:__ DAGs are dynamic in PyTorch An important thing to note is that the graph is recreated from scratch; after each <code>.backward()</code> call, autograd starts populating a new graph. This is exactly what allows you to use control flow statements in your model; you can change the shape, size and operations at every iteration if needed.

__Tensor Gradients and Jacobian Products__

In many cases, we have a scalar loss function, and we need to compute the gradient with respect to some parameters. However, there are cases when the output function is an arbitrary tensor. In this case, PyTorch allows you to compute so-called __Jacobian product__, and not the actual gradient.

![jacobian](imgs/jacobian.png)

In [7]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Notice that when we call backward for the second time with the same argument, the value of the gradient is different. This happens because when doing backward propagation, PyTorch accumulates the gradients, i.e. the value of computed gradients is added to the grad property of all leaf nodes of computational graph. If you want to compute the proper gradients, you need to zero out the grad property before. In real-life training an optimizer helps us to do this.

__Note:__ Previously we were calling <code>backward()</code> function without parameters. This is essentially equivalent to calling <code>backward(torch.tensor(1.0))</code>, which is a useful way to compute the gradients in case of a scalar-valued function, such as loss during neural network training.